Libraries

In [80]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import os
import polars as pl

File Enumeration and filtering:

In [81]:
def filter_and_append_to_csv(input_file, output_file, product):
    try:
        df_input = pd.read_csv(input_file, on_bad_lines='skip')
        
        print("First few rows of the input DataFrame:")
        display(df_input.head())
        
        filtered_df = df_input[df_input.iloc[:, 0].str.contains(product, na=False)]
        
        print(f"Number of rows in filtered DataFrame: {len(filtered_df)}")
        
        # Write to output file, append if exists
        if not os.path.exists(output_file):
            filtered_df.to_csv(output_file, index=False)
        else:
            filtered_df.to_csv(output_file, mode='a', header=False, index=False)

        return filtered_df

    except Exception as e:
        print(f"Error processing {input_file}: {str(e)}")
        return None

def enum():
    folder_path = os.getcwd()
    print("Enumerating files...")
    
    csv_files = [file for file in os.listdir(folder_path) if file.startswith('rnaseq_data')]
    
    renamed_files = []
    # Rename each file with a numbered format
    for index, file in enumerate(csv_files, start=1):
        old_path = os.path.join(folder_path, file)
        new_name = f'{index}.csv'
        new_path = os.path.join(folder_path, new_name)
        os.rename(old_path, new_path)
        renamed_files.append(new_name)
    
    print("Files have been successfully renamed.")
    return renamed_files


testing enum:

In [82]:
renamed_files = enum()
print("Renamed files:", renamed_files)

Enumerating files...
Files have been successfully renamed.
Renamed files: []


Testing filter_append_to_Csv:

In [83]:
if renamed_files:
    test_input_file = renamed_files[0]  # Use the first renamed file
    test_output_file = "output_file.csv"
    test_product = "methanol dehydrogenase (cytochrome c) subunit 1"  # Change this to a product you want to filter by

    filtered_df = filter_and_append_to_csv(test_input_file, test_output_file, test_product)

    if filtered_df is not None and not filtered_df.empty:
        display(filtered_df)
    else:
        print("No data to display.")
else:
    print("No files were renamed.")

No files were renamed.


Linkgen function: 

In [84]:
def linkgen():
    try:
        filename = "output_file.csv"
        df = pl.read_csv(filename)
        df = df.with_columns(pl.col("Gene seq length").str.concat([
            "https://img.jgi.doe.gov/cgi-bin/mer/main.cgi?section=MetaGeneDetail&page=genePageMainFaa&taxon_oid=", 
            df["Gene seq length"], 
            "&data_type=assembled&gene_oid=", 
            df["Gene seq length"].str.split(",").arr.first()
        ], sep='').alias("Link"))

        df.select(["Gene seq length", "Link"]).write_csv("Link_Output.csv")
        print("Done generating links.")
    except Exception as e:
        print(f"Error generating links: {str(e)}")

In [85]:
linkgen()

Error generating links: Gene seq length
